In [ ]:
import requests
import requests.auth
import pandas as pd
import json
import re

In [ ]:

f = json.load(open('C:/Users/elima/Desktop/Projects/Hinge Project/keys'))

client_ID = f['reddit_api_key2']['client_ID']
secret_key = f['reddit_api_key2']['secret_key']

auth = requests.auth.HTTPBasicAuth(client_ID, secret_key)
data = {
    'grant_type': 'password',
    'username': f['reddit_login']['user_name'],
    'password': f['reddit_login']['password']
}

headers = {'User-Agent': 'api_ver_1'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
access_token = res.json()['access_token']

In [ ]:
headers['Authorization'] = 'bearer ' + access_token
base_url = 'https://oauth.reddit.com'

In [ ]:
df = pd.read_csv('C:/Users/elima/Desktop/Projects/Hinge Project/HingeDataFramePosts')
pd.set_option('display.max_colwidth', None)

In [ ]:
comments_payload = {'depth':0, 'limit':10}
list_of_comments = []

for i in range(len(df)):
    post_ID = df['post id'][i]
    profile_review_post = requests.get(base_url + '/r/hingeapp/comments/'+ post_ID, headers=headers, params=comments_payload).json()
    id_and_top_comments = {}; j = 1
    id_and_top_comments['post id'] = post_ID;
    for comment in profile_review_post[1]['data']['children'][1:]:
        if ('body' in comment['data']) and (comment['data']['author']!='AutoModerator'):
            id_and_top_comments[f'comment {j}'] = comment['data']['body']
        j += 1
    list_of_comments.append(id_and_top_comments)

In [ ]:
df_comments = pd.DataFrame(list_of_comments)
pd.set_option('display.max_colwidth', None)

In [ ]:
def clean_text(text):
    pattern = '[^a-zA-Z0-9\s]'
    cleaned_text = re.sub(pattern, '', text).replace('\n', ' ').lower()
    return cleaned_text

for column in df_comments.columns[1:]:
    df_comments[column] = df_comments[column].apply(lambda x: clean_text(x) if pd.notnull(x) else x) 

In [ ]:
df_comments.to_csv('C:/Users/elima/Desktop/Projects/Hinge Project/HingeDataFrameComments')
df_comments